## Test rf

In [2]:
import unittest
import pandas as pd
import numpy as np
import ease

class TEST_RF(unittest.TestCase):
    
    def test_rf_inputtrain(self):
        train = pd.read_csv('../Arranged_Data/final_weater.csv')[[
            'State', 'TotalMonthlyPrecip', 'TempSummer',
            'TempWinter', 'Avgwindspeed']]
        #test if DC exists
        self.assertIn('DC',list(train.State))
        #test if there is nan in input data
        self.assertEqual(train.isnull().sum().sum(),0)
        
    def test_rf_inputpara(self):
        test = pd.read_csv('../Arranged_Data/test_dataset.csv')
        prec = test.iloc[0,1]
        ts = test.iloc[0,2]
        tw = test.iloc[0,3]
        ws = test.iloc[0,4]
        #test the type of input data
        self.assertIsInstance(prec,np.float64)
        self.assertIsInstance(ts,np.float64)
        self.assertIsInstance(tw,np.float64)
        self.assertIsInstance(ws,np.float64)
        #test the limitation of input temperature
        self.assertGreater(110, ts)
        self.assertGreater(ts, tw)
        self.assertGreater(tw,-10)
        #test the limitation of input precipitation
        self.assertGreater(prec, 0)
        #test the limitation of input windspeed
        self.assertGreater(ws, 0)
    
    def test_rf_output(self):
        test = pd.read_csv('../Arranged_Data/test_dataset.csv')
        prec = test.iloc[0,1]
        ts = test.iloc[0,2]
        tw = test.iloc[0,3]
        ws = test.iloc[0,4]
        output = ease.rf(prec, ts, tw, ws)
        #test the type of output dataset
        self.assertIsInstance(output,dict)
        #test the output values
        for i in list(output.values()):
            self.assertTrue(i < 1)
            self.assertTrue(i > 0)
                
if __name__ == '__main__':
    unittest.main()

## Test avg_capacity

In [1]:
import unittest
import pandas as pd
import numpy as np
import ease

class TEST_avg_capacity(unittest.TestCase):
    
    def test_avg_cap_input(self):
        average_plant_capacity = pd.read_csv('../Arranged_Data/average_plant_capacity.csv')
        #test if there is nan in input dataframe
        self.assertEqual(average_plant_capacity.isnull().sum().sum(),0)
    
    def test_avg_cap_output(self):
        average_plant_capacity = pd.read_csv('../Arranged_Data/average_plant_capacity.csv')
        test = pd.read_csv('../Arranged_Data/test_dataset.csv')
        prec = test.iloc[0,1]
        ts = test.iloc[0,2]
        tw = test.iloc[0,3]
        ws = test.iloc[0,4]
        vote = ease.rf(prec, ts, tw, ws)
        output = ease.avg_capacity(vote)
        #test the type of output dataset
        self.assertIsInstance(output,list)
        #test the length of output dataset
        self.assertEqual(len(output),6) 
if __name__ == '__main__':
    unittest.main()

In [11]:
vote = ease.rf(3, 69, 35, 5.4)
ease.avg_capacity(vote)

[2551245.8119999995,
 286664.9808,
 62748.209599999995,
 64409.339799999994,
 70.3428,
 129394.76900000001]

## Test possible_type

In [4]:
avg_cap_list = [2505375.75, 287825.37, 62167.29, 56935.33, 89.96000000000001, 126547.58]
a = ease.possible_type(avg_cap_list)

In [7]:
import ease
ease.possible_type(avg_cap_list)[1][-1]

'NG'

In [4]:
import unittest
import pandas as pd
import numpy as np
import ease

class TEST_possible_type(unittest.TestCase):

    def test_possible_type_input(self):
        cap_pop = pd.read_csv('../Arranged_Data/average_plant_capacity.csv')
        #test if there is nan in input capacity dataframe
        self.assertEqual(cap_pop.isnull().sum().sum(),0)
        
    def test_possible_type_output(self):
        average_plant_capacity = pd.read_csv('../Arranged_Data/average_plant_capacity.csv')
        test = pd.read_csv('../Arranged_Data/test_dataset.csv')
        prec = test.iloc[0,1]
        ts = test.iloc[0,2]
        tw = test.iloc[0,3]
        ws = test.iloc[0,4]
        vote = ease.rf(prec, ts, tw, ws)
        avg_cap_list = ease.avg_capacity(vote)
        output = ease.possible_type(avg_cap_list)
        
        #test the type of output dataset
        self.assertIsInstance(output,list)
        
        #calculate how many clean energy resource have been filtered
        type_list = []
        for i in range(len(ease.possible_type(avg_cap_list))):
            type = ease.possible_type(avg_cap_list)[i][-1]
            type_list.append(type)
            clean_ = ['Hydro', 'Solar', 'Wind']
        filtered = []
        for i in clean_:
            if i in type_list:
                pass
            else:
                filtered.append(i)
                
        #test if all the clean resources have been filtered out
        self.assertNotEqual(len(filtered),3)

if __name__ == '__main__':
    unittest.main()

## Test rf_fluctuation

In [7]:
import unittest
import pandas as pd
import numpy as np
import ease

class TEST_rf_fluctuation(unittest.TestCase):
    
    def test_rf_fluctuation_output(self):
        test = pd.read_csv('../Arranged_Data/test_dataset.csv')
        prec = test.iloc[0,1]
        ts = test.iloc[0,2]
        tw = test.iloc[0,3]
        ws = test.iloc[0,4]
        output = ease.rf_fluctuation(prec, ts, tw, ws)
        
        #test the type of output
        self.assertIsInstance(output,list)
        #test the length of output
        self.assertEqual(len(output),3) 
        #test fluctuation level
        std = output[1]
        self.assertLess(std, 0.01)

if __name__ == '__main__':
    unittest.main()